# Mais sobre `SELECT`

- temporary tables
- views
- variables
- subqueries


In [1]:
from functools import partial
from dotenv import load_dotenv
import insperautograder.jupyter as ia
import mysql.connector
import os

load_dotenv(override=True)

connection = mysql.connector.connect(
    host=os.getenv("MD_DB_SERVER"),
    user=os.getenv("MD_DB_USERNAME"),
    password=os.getenv("MD_DB_PASSWORD"),
    database="sakila"
)

def run_db_query(connection, query, args=None):
    with connection.cursor() as cursor:
        print("Executando query:")
        cursor.execute(query, args)
        for result in cursor:
            print(result)

db = partial(run_db_query, connection)

## Exercícios para entrega

Esta aula tem atividade para entrega, confira os prazos e exercícios

In [2]:
ia.tasks()

|    | Atividade    | De                               | Até                       |
|---:|:-------------|:---------------------------------|:--------------------------|
|  0 | newborn      | 2023-08-08 03:00:00+00:00        | 2023-08-16 02:59:59+00:00 |
|  1 | select01     | 2023-08-08 03:00:00+00:00        | 2023-08-21 02:59:59+00:00 |
|  2 | ddl          | 2023-08-27 20:36:25.452000+00:00 | 2023-09-02 02:59:59+00:00 |
|  3 | dml          | 2023-08-29 20:36:25.452000+00:00 | 2023-09-04 02:59:59+00:00 |
|  4 | agg_join     | 2023-09-03 03:00:00+00:00        | 2023-09-09 02:59:59+00:00 |
|  5 | group_having | 2023-09-03 03:00:00+00:00        | 2023-09-17 02:59:59+00:00 |
|  6 | views        | 2023-09-11 03:00:00+00:00        | 2023-09-17 02:59:59+00:00 |
|  7 | sql_review1  | 2023-09-13 03:00:00+00:00        | 2023-09-20 02:59:59+00:00 |

In [3]:
ia.grades(by="task")

|    | Tarefa       |     Nota |
|---:|:-------------|---------:|
|  0 | agg_join     | 10       |
|  1 | ddl          | 10       |
|  2 | dml          | 10       |
|  3 | group_having | 10       |
|  4 | newborn      | 10       |
|  5 | select01     | 10       |
|  6 | sql_review1  |  0       |
|  7 | views        |  8.33333 |

In [4]:
ia.grades(task="views")

|    | Atividade   | Exercício   |   Peso |   Nota |
|---:|:------------|:------------|-------:|-------:|
|  0 | views       | ex01        |      1 |     10 |
|  1 | views       | ex02        |      1 |     10 |
|  2 | views       | ex03        |      1 |     10 |
|  3 | views       | ex04        |      1 |     10 |
|  4 | views       | ex05        |      1 |     10 |
|  5 | views       | ex06        |      1 |     10 |
|  6 | views       | ex07        |      1 |     10 |
|  7 | views       | ex08        |      1 |     10 |
|  8 | views       | ex09        |      1 |     10 |
|  9 | views       | ex10        |      1 |     10 |
| 10 | views       | ex11        |      1 |      0 |
| 11 | views       | ex12        |      1 |      0 |

## Aquecimento

Quanta receita foi gerada para cada categoria de filmes? Mostre o nome da categoria e a receita. Ordene da maior receita para para a menor.

In [5]:
sql_ex01 = '''
SELECT 
    category.name, SUM(payment.amount) AS total
FROM
    payment
    INNER JOIN customer USING (customer_id)
    INNER JOIN rental USING (customer_id)
    INNER JOIN inventory USING (inventory_id)
    INNER JOIN film USING (film_id)
    INNER JOIN film_category USING (film_id)
    INNER JOIN category USING (category_id)
GROUP BY
    category.category_id
ORDER BY
    total DESC
'''

db(sql_ex01)

Executando query:
('Sports', Decimal('138259.63'))
('Animation', Decimal('137087.62'))
('Action', Decimal('130658.86'))
('Family', Decimal('129027.81'))
('Sci-Fi', Decimal('127759.42'))
('Drama', Decimal('122038.34'))
('Documentary', Decimal('122030.27'))
('Foreign', Decimal('120764.57'))
('Games', Decimal('114440.09'))
('New', Decimal('110423.76'))
('Comedy', Decimal('110367.36'))
('Children', Decimal('109711.51'))
('Classics', Decimal('108909.89'))
('Horror', Decimal('98369.72'))
('Travel', Decimal('97278.10'))
('Music', Decimal('95825.55'))


In [6]:
ia.sender(answer='sql_ex01', task='views', question='ex01', answer_type='pyvar')

interactive(children=(Button(description='Enviar ex01', style=ButtonStyle()), Output()), _dom_classes=('widget…

Liste os 10 filmes mais alugados e o numero de vezes em que foram alugados. Ordene do mais alugado para o menos alugado.

In [7]:
sql_ex02 = '''
SELECT 
    film.title, COUNT(rental.rental_id) AS total_rentals
FROM
    rental
    INNER JOIN inventory USING (inventory_id)
    INNER JOIN film USING (film_id)
GROUP BY
    film.film_id
ORDER BY
    total_rentals DESC
LIMIT 10
'''

db(sql_ex02)

Executando query:
('BUCKET BROTHERHOOD', 34)
('ROCKETEER MOTHER', 33)
('RIDGEMONT SUBMARINE', 32)
('GRIT CLOCKWORK', 32)
('SCALAWAG DUCK', 32)
('JUGGLER HARDLY', 32)
('FORWARD TEMPLE', 32)
('HOBBIT ALIEN', 31)
('ROBBERS JOON', 31)
('ZORRO ARK', 31)


In [8]:
ia.sender(answer='sql_ex02', task='views', question='ex02', answer_type='pyvar')

interactive(children=(Button(description='Enviar ex02', style=ButtonStyle()), Output()), _dom_classes=('widget…

**DESAFIO:** Liste os filmes e o numero de vezes em que foram alugados apenas para filmes que foram alugados mais do que a média de numero de alugueis por filme. *Dica*: Serão necessários múltiplos SELECTs. Faça cada um em uma célula diferente.

In [9]:
db('''
SELECT 
    AVG(total_rentals)
INTO @avg_rentals
FROM
    (SELECT
        COUNT(rental.rental_id) AS total_rentals
    FROM
        rental
        INNER JOIN inventory USING (inventory_id)
        INNER JOIN film USING (film_id)
    GROUP BY
        film.film_id
    ) AS rentals
''')
db('''
SELECT @avg_rentals
''')
db('''
SELECT
    film.title, COUNT(rental.rental_id) AS total_rentals
FROM
    rental
    INNER JOIN inventory USING (inventory_id)
    INNER JOIN film USING (film_id)
GROUP BY
    film.film_id
HAVING
    total_rentals > @avg_rentals
ORDER BY
    total_rentals
''')

Executando query:
Executando query:
(Decimal('16.747390396'),)
Executando query:
('BEACH HEARTBREAKERS', 17)
('BIKINI BORROWERS', 17)
('BREAKFAST GOLDFINGER', 17)
('COMFORTS RUSH', 17)
('CONFUSED CANDLES', 17)
('CONVERSATION DOWNHILL', 17)
('DYING MAKER', 17)
('EGYPT TENENBAUMS', 17)
('FACTORY DRAGON', 17)
('FERRIS MOTHER', 17)
('FLINTSTONES HAPPINESS', 17)
('FREAKY POCUS', 17)
('FRENCH HOLIDAY', 17)
('GANDHI KWAI', 17)
('GREEK EVERYONE', 17)
('HANOVER GALAXY', 17)
('HOLOCAUST HIGHBALL', 17)
('IDAHO LOVE', 17)
('INTENTIONS EMPIRE', 17)
('JEKYLL FROGMEN', 17)
('JERICHO MULAN', 17)
('JOON NORTHWEST', 17)
('LADY STAGE', 17)
('LAWLESS VISION', 17)
('LEAGUE HELLFIGHTERS', 17)
('LOUISIANA HARRY', 17)
('MAJESTIC FLOATS', 17)
('MASK PEACH', 17)
('MIDNIGHT WESTWARD', 17)
('MUSKETEERS WAIT', 17)
('NEWTON LABYRINTH', 17)
('NOON PAPI', 17)
('ORIENT CLOSER', 17)
('PERFECT GROOVE', 17)
('POSEIDON FOREVER', 17)
('POTTER CONNECTICUT', 17)
('REAR TRADING', 17)
('ROCK INSTINCT', 17)
('ROOTS REMEMBER', 1

## Views

Uma *view* é uma tabela virtual, construida a partir de um comando `SELECT`. Por exemplo: execute o código a seguir.

In [10]:
db('''
DROP VIEW IF EXISTS movie_count;
''')

db('''
CREATE VIEW movie_count AS
    SELECT 
        title, COUNT(rental_id) as cnt
    FROM
        film
        LEFT OUTER JOIN inventory USING (film_id)
        LEFT OUTER JOIN rental USING (inventory_id)
    GROUP BY
        film_id
    ORDER BY
        cnt ASC;
''')

Executando query:
Executando query:


Agora temos uma *view* chamada `movie_count`. Vamos verificar que ela funcionou, listando as 30 primeiras linhas:

In [11]:
db('''
SELECT 
    *
FROM
    movie_count 
LIMIT 
    30
''')

Executando query:
('ALICE FANTASIA', 0)
('APOLLO TEEN', 0)
('ARGONAUTS TOWN', 0)
('ARK RIDGEMONT', 0)
('ARSENIC INDEPENDENCE', 0)
('BOONDOCK BALLROOM', 0)
('BUTCH PANTHER', 0)
('CATCH AMISTAD', 0)
('CHINATOWN GLADIATOR', 0)
('CHOCOLATE DUCK', 0)
('COMMANDMENTS EXPRESS', 0)
('CROSSING DIVORCE', 0)
('CROWDS TELEMARK', 0)
('CRYSTAL BREAKING', 0)
('DAZED PUNK', 0)
('DELIVERANCE MULHOLLAND', 0)
('FIREHOUSE VIETNAM', 0)
('FLOATS GARDEN', 0)
('FRANKENSTEIN STRANGER', 0)
('GLADIATOR WESTWARD', 0)
('GUMP DATE', 0)
('HATE HANDICAP', 0)
('HOCUS FRIDA', 0)
('KENTUCKIAN GIANT', 0)
('KILL BROTHERHOOD', 0)
('MUPPET MILE', 0)
('ORDER BETRAYED', 0)
('PEARL DESTINY', 0)
('PERDITION FARGO', 0)
('PSYCHO SHRUNK', 0)


In [12]:
db('''
SELECT 
    * 
FROM 
    movie_count 
ORDER BY
    cnt DESC
LIMIT 30
''')

Executando query:
('BUCKET BROTHERHOOD', 34)
('ROCKETEER MOTHER', 33)
('SCALAWAG DUCK', 32)
('RIDGEMONT SUBMARINE', 32)
('JUGGLER HARDLY', 32)
('GRIT CLOCKWORK', 32)
('FORWARD TEMPLE', 32)
('ZORRO ARK', 31)
('WIFE TURN', 31)
('TIMBERLAND SKY', 31)
('RUSH GOODFELLAS', 31)
('ROBBERS JOON', 31)
('NETWORK PEAK', 31)
('HOBBIT ALIEN', 31)
('GOODFELLAS SALUTE', 31)
('APACHE DIVINE', 31)
('WITCHES PANIC', 30)
('SUSPECTS QUILLS', 30)
('SHOCK CABIN', 30)
('RUGRATS SHAKESPEARE', 30)
('PULP BEVERLY', 30)
('MUSCLE BRIGHT', 30)
('MASSACRE USUAL', 30)
('MARRIED GO', 30)
('IDOLS SNATCHERS', 30)
('HARRY IDAHO', 30)
('GRAFFITI LOVE', 30)
('FROST HEAD', 30)
('ENGLISH BULWORTH', 30)
('DOGMA FAMILY', 30)


Agora suponha que alteramos a tabela `film`, mudando o nome do filme "DAZED PUNK" para "STONED PUNK".

**Atividade**: Do it.

In [13]:
db('''
UPDATE
    film
SET
    title = 'STONED PUNK'
WHERE
    title = 'DAZED PUNK'
''')

Executando query:


Verifique agora a nossa *view*:

In [14]:
db('''
SELECT
    *
FROM
    movie_count
LIMIT
    30
''')

Executando query:
('ALICE FANTASIA', 0)
('APOLLO TEEN', 0)
('ARGONAUTS TOWN', 0)
('ARK RIDGEMONT', 0)
('ARSENIC INDEPENDENCE', 0)
('BOONDOCK BALLROOM', 0)
('BUTCH PANTHER', 0)
('CATCH AMISTAD', 0)
('CHINATOWN GLADIATOR', 0)
('CHOCOLATE DUCK', 0)
('COMMANDMENTS EXPRESS', 0)
('CROSSING DIVORCE', 0)
('CROWDS TELEMARK', 0)
('CRYSTAL BREAKING', 0)
('STONED PUNK', 0)
('DELIVERANCE MULHOLLAND', 0)
('FIREHOUSE VIETNAM', 0)
('FLOATS GARDEN', 0)
('FRANKENSTEIN STRANGER', 0)
('GLADIATOR WESTWARD', 0)
('GUMP DATE', 0)
('HATE HANDICAP', 0)
('HOCUS FRIDA', 0)
('KENTUCKIAN GIANT', 0)
('KILL BROTHERHOOD', 0)
('MUPPET MILE', 0)
('ORDER BETRAYED', 0)
('PEARL DESTINY', 0)
('PERDITION FARGO', 0)
('PSYCHO SHRUNK', 0)


Como você pode ver, as views são tabelas virtuais que são automaticamente atualizadas quando as tabelas originais são modificadas.

In [15]:
connection.rollback()

In [16]:
db('''
SELECT
    *
FROM
    movie_count
LIMIT
    30
''')

Executando query:
('ALICE FANTASIA', 0)
('APOLLO TEEN', 0)
('ARGONAUTS TOWN', 0)
('ARK RIDGEMONT', 0)
('ARSENIC INDEPENDENCE', 0)
('BOONDOCK BALLROOM', 0)
('BUTCH PANTHER', 0)
('CATCH AMISTAD', 0)
('CHINATOWN GLADIATOR', 0)
('CHOCOLATE DUCK', 0)
('COMMANDMENTS EXPRESS', 0)
('CROSSING DIVORCE', 0)
('CROWDS TELEMARK', 0)
('CRYSTAL BREAKING', 0)
('DAZED PUNK', 0)
('DELIVERANCE MULHOLLAND', 0)
('FIREHOUSE VIETNAM', 0)
('FLOATS GARDEN', 0)
('FRANKENSTEIN STRANGER', 0)
('GLADIATOR WESTWARD', 0)
('GUMP DATE', 0)
('HATE HANDICAP', 0)
('HOCUS FRIDA', 0)
('KENTUCKIAN GIANT', 0)
('KILL BROTHERHOOD', 0)
('MUPPET MILE', 0)
('ORDER BETRAYED', 0)
('PEARL DESTINY', 0)
('PERDITION FARGO', 0)
('PSYCHO SHRUNK', 0)


### Vamos praticar

Verifique quantas vezes o filme "COWBOY DOOM" foi alugado usando a view `movie_count`

In [17]:
sql_ex03 = '''
SELECT
    cnt
FROM
    movie_count
WHERE
    title = 'COWBOY DOOM'
'''

db(sql_ex03)

Executando query:
(7,)


In [18]:
ia.sender(answer='sql_ex03', task='views', question='ex03', answer_type='pyvar')

interactive(children=(Button(description='Enviar ex03', style=ButtonStyle()), Output()), _dom_classes=('widget…

Registre um aluguel do filme "COWBOY DOOM" (com o menor id possível), feito pelo funcionario "Jon Stephens" na loja id=1 para o cliente "JESSIE BANKS", na data '2019-01-01', com data de retorno '2019-01-08'.

In [19]:
db('SELECT rental_id FROM rental ORDER BY rental_id DESC LIMIT 1')

Executando query:
(16049,)


In [20]:
db("SELECT staff_id FROM staff WHERE first_name = 'Jon' AND last_name = 'Stephens'")

Executando query:
(2,)


In [21]:
db("SELECT customer_id FROM customer WHERE first_name = 'JESSIE' AND last_name = 'BANKS'")

Executando query:
(215,)


In [22]:
db("SELECT inventory_id FROM inventory WHERE film_id = (SELECT film_id FROM film WHERE title = 'COWBOY DOOM') LIMIT 1")

Executando query:
(845,)


In [23]:
sql_ex04 = '''
INSERT INTO
    rental
VALUES (
    0,
    '2023-09-11 12:42:07',
    (SELECT inventory_id FROM inventory WHERE film_id = (SELECT film_id FROM film WHERE title = 'COWBOY DOOM') LIMIT 1),
    (SELECT customer_id FROM customer WHERE first_name = 'JESSIE' AND last_name = 'BANKS'),
    '2023-01-08 00:00:00',
    (SELECT staff_id FROM staff WHERE first_name = 'Jon' AND last_name = 'Stephens'),
    current_date
)
'''

db(sql_ex04)

Executando query:


In [24]:
ia.sender(answer='sql_ex04', task='views', question='ex04', answer_type='pyvar')

interactive(children=(Button(description='Enviar ex04', style=ButtonStyle()), Output()), _dom_classes=('widget…

Verifique usando a view que a contagem de alugueis do filme subiu.

In [25]:
db('''
SELECT
    *
FROM
    movie_count
WHERE
    title = 'COWBOY DOOM'
''')

Executando query:
('COWBOY DOOM', 8)


Faça o *rollback* desta alteração, para não modificar a nossa querida base de dados *sakila*.

In [26]:
connection.rollback()

## Tabelas temporárias

Tabelas temporárias podem ser criadas para ajudar nas tarefas de manipulação de dados. Essas tabelas existem apenas pela duração da sessão. Para criar uma tabela temporária, basta adicionar a palavra-chave `TEMPORARY` no momento da criação.

É comum criar tabelas temporárias à partir do resultado de comandos `SELECT`. Por exemplo, a seguinte query permite montar uma tabela temporária com os filmes que duram mais que 3 horas:

In [27]:
db('''
DROP TABLE IF EXISTS long_film
''')

db('''
CREATE TEMPORARY TABLE long_film 
    SELECT
        *
    FROM
        film
    WHERE
        film.length > 180;
''')

Executando query:
Executando query:


Podemos verificar que a tabela `long_film` agora existe:

In [28]:
db('DESCRIBE long_film')

Executando query:
('film_id', 'smallint unsigned', 'NO', '', '0', 'NULL')
('title', 'varchar(128)', 'NO', '', None, 'NULL')
('description', 'text', 'YES', '', None, 'NULL')
('release_year', 'year', 'YES', '', None, 'NULL')
('language_id', 'tinyint unsigned', 'NO', '', None, 'NULL')
('original_language_id', 'tinyint unsigned', 'YES', '', None, 'NULL')
('rental_duration', 'tinyint unsigned', 'NO', '', '3', 'NULL')
('rental_rate', 'decimal(4,2)', 'NO', '', '4.99', 'NULL')
('length', 'smallint unsigned', 'YES', '', None, 'NULL')
('replacement_cost', 'decimal(5,2)', 'NO', '', '19.99', 'NULL')
('rating', "enum('G','PG','PG-13','R','NC-17')", 'YES', '', 'G', 'NULL')
('special_features', "set('Trailers','Commentaries','Deleted Scenes','Behind the Scenes')", 'YES', '', None, 'NULL')
('last_update', 'timestamp', 'NO', '', 'CURRENT_TIMESTAMP', 'on update CURRENT_TIMESTAMP')


In [29]:
db("CALL sys.table_exists('sakila', 'long_film', @table_type);")
db("SELECT @table_type;")

Executando query:
Executando query:
('TEMPORARY',)


Muito embora ela não apareça na lista de tabelas: isso é um bug do MySQL. (https://dev.mysql.com/worklog/task/?id=648)

In [30]:
db('SHOW TABLES')

Executando query:
('actor',)
('actor_info',)
('address',)
('category',)
('city',)
('country',)
('customer',)
('customer_list',)
('film',)
('film_actor',)
('film_category',)
('film_list',)
('film_text',)
('inventory',)
('language',)
('movie_count',)
('nicer_but_slower_film_list',)
('payment',)
('rental',)
('sales_by_film_category',)
('sales_by_store',)
('staff',)
('staff_list',)
('store',)


Vamos listar o conteudo desta tabela:

In [31]:
db('SELECT title FROM long_film')

Executando query:
('ANALYZE HOOSIERS',)
('BAKED CLEOPATRA',)
('CATCH AMISTAD',)
('CHICAGO NORTH',)
('CONSPIRACY SPIRIT',)
('CONTROL ANTHEM',)
('CRYSTAL BREAKING',)
('DARN FORRESTER',)
('FRONTIER CABIN',)
('GANGS PRIDE',)
('HAUNTING PIANIST',)
('HOME PITY',)
('HOTEL HAPPINESS',)
('INTRIGUE WORST',)
('JACKET FRISCO',)
('KING EVOLUTION',)
('LAWLESS VISION',)
('LOVE SUICIDES',)
('MONSOON CAUSE',)
('MOONWALKER FOOL',)
('MUSCLE BRIGHT',)
('POND SEATTLE',)
('RECORDS ZORRO',)
('REDS POCUS',)
('RUNAWAY TENENBAUMS',)
('SATURN NAME',)
('SCALAWAG DUCK',)
('SEARCHERS WAIT',)
('SMOOCHY CONTROL',)
('SOLDIERS EVOLUTION',)
('SONS INTERVIEW',)
('SORORITY QUEEN',)
('STAR OPERATION',)
('SWEET BROTHERHOOD',)
('THEORY MERMAID',)
('WIFE TURN',)
('WILD APOLLO',)
('WORST BANGER',)
('YOUNG LANGUAGE',)


Vamos apagar a tabela `long_film`:

In [32]:
db('DROP TABLE long_film')

Executando query:


### Vamos praticar

- Crie uma tabela temporária `max_duration` que contém a duração máxima de filme para cada categoria. Apresente o id da categoria, seu nome e a duração máxima.

In [33]:
db('DROP TABLE IF EXISTS max_duration')

sql_ex05 = '''
CREATE TEMPORARY TABLE max_duration
    SELECT
        category_id, name, MAX(film.length) as max_len
    FROM
        film
        INNER JOIN film_category USING (film_id)
        INNER JOIN category USING (category_id)
    GROUP BY
        category_id
'''

db(sql_ex05)

Executando query:
Executando query:


In [34]:
ia.sender(answer='sql_ex05', task='views', question='ex05', answer_type='pyvar')

interactive(children=(Button(description='Enviar ex05', style=ButtonStyle()), Output()), _dom_classes=('widget…

 - Verifique a tabela.

In [35]:
db('SELECT * FROM max_duration')

Executando query:
(1, 'Action', 185)
(2, 'Animation', 185)
(3, 'Children', 178)
(4, 'Classics', 184)
(5, 'Comedy', 185)
(6, 'Documentary', 183)
(7, 'Drama', 181)
(8, 'Family', 184)
(9, 'Foreign', 184)
(10, 'Games', 185)
(11, 'Horror', 181)
(12, 'Music', 185)
(13, 'New', 183)
(14, 'Sci-Fi', 185)
(15, 'Sports', 184)
(16, 'Travel', 185)


In [36]:
db('DESCRIBE max_duration')

Executando query:
('category_id', 'tinyint unsigned', 'NO', '', None, 'NULL')
('name', 'varchar(25)', 'NO', '', None, 'NULL')
('max_len', 'smallint unsigned', 'YES', '', None, 'NULL')


- Agora use a tabela temporária para construir uma consulta com as categorias e seus respectivos filmes mais longos:

In [37]:
db('''
SELECT
    category_id, name, film_id, title, length
FROM
    film
    INNER JOIN film_category USING (film_id)
    INNER JOIN max_duration USING (category_id)
WHERE
    length = max_len
''')

Executando query:
(1, 'Action', 212, 'DARN FORRESTER', 185)
(1, 'Action', 991, 'WORST BANGER', 185)
(2, 'Animation', 349, 'GANGS PRIDE', 185)
(2, 'Animation', 690, 'POND SEATTLE', 185)
(3, 'Children', 344, 'FURY MURDER', 178)
(3, 'Children', 993, 'WRONG BEHAVIOR', 178)
(4, 'Classics', 180, 'CONSPIRACY SPIRIT', 184)
(5, 'Comedy', 182, 'CONTROL ANTHEM', 185)
(6, 'Documentary', 973, 'WIFE TURN', 183)
(6, 'Documentary', 996, 'YOUNG LANGUAGE', 183)
(7, 'Drama', 473, 'JACKET FRISCO', 181)
(8, 'Family', 499, 'KING EVOLUTION', 184)
(9, 'Foreign', 198, 'CRYSTAL BREAKING', 184)
(9, 'Foreign', 821, 'SORORITY QUEEN', 184)
(10, 'Games', 141, 'CHICAGO NORTH', 185)
(11, 'Horror', 24, 'ANALYZE HOOSIERS', 181)
(11, 'Horror', 535, 'LOVE SUICIDES', 181)
(12, 'Music', 426, 'HOME PITY', 185)
(13, 'New', 340, 'FRONTIER CABIN', 183)
(14, 'Sci-Fi', 817, 'SOLDIERS EVOLUTION', 185)
(15, 'Sports', 813, 'SMOOCHY CONTROL', 184)
(16, 'Travel', 609, 'MUSCLE BRIGHT', 185)
(16, 'Travel', 872, 'SWEET BROTHERHOOD', 185)

- delete a tabela temporária

In [38]:
db('''
DROP TABLE max_duration
''')

Executando query:


## Variáveis

Podemos montar uma query que retorne um valor só e armazenar este valor em uma variável, para uso posterior em outras queries. Para isso vamos usar o prefixo '@' para indicar variáveis, e o comando `SELECT ... INTO`.

Exemplo: quais são os filmes "caros" da nossa base sakila? Vamos descobrir quais filmes custam mais que um desvio padrão acima da média de preços de locação.

Primeiro vamos calcular a média e o desvio padrão dos preços de aluguel:

In [39]:
db('''
SELECT 
    AVG(rental_rate), 
    STDDEV(rental_rate)
INTO 
    @avg_rate, 
    @stddev_rate 
FROM
    film;
''')

Executando query:


Note que a query não retorna um resultado: o resultado foi armazenado direto nas variáveis `@avg_rate` e `@stddev_rate`. Vamos usar um `SELECT` sem tabelas para ver o resultado:

In [40]:
db('SELECT @avg_rate, @stddev_rate')

Executando query:
(Decimal('2.980000000'), 1.6455698101265719)


Agora podemos selecionar os filmes caros!

In [41]:
db('''
SELECT 
    title, rental_rate
FROM
    film
WHERE
    rental_rate > @avg_rate + @stddev_rate
LIMIT 10
''')

Executando query:
('ACE GOLDFINGER', Decimal('4.99'))
('AIRPLANE SIERRA', Decimal('4.99'))
('AIRPORT POLLOCK', Decimal('4.99'))
('ALADDIN CALENDAR', Decimal('4.99'))
('ALI FOREVER', Decimal('4.99'))
('AMELIE HELLFIGHTERS', Decimal('4.99'))
('AMERICAN CIRCUS', Decimal('4.99'))
('ANTHEM LUKE', Decimal('4.99'))
('APACHE DIVINE', Decimal('4.99'))
('APOCALYPSE FLAMINGOS', Decimal('4.99'))


### Vamos praticar

Armazene na variável temporária `most_films` a quantidade de filmes feitos pelo ator ou atriz que mais participou de filmes.

In [42]:
sql_ex06 = '''
SELECT
    COUNT(film.film_id)
INTO
    @most_films
FROM
    film
    INNER JOIN film_actor USING (film_id)
    INNER JOIN actor USING (actor_id)
GROUP BY
    actor_id
ORDER BY
    COUNT(film.film_id) DESC
LIMIT 1
'''

db(sql_ex06)

Executando query:


In [43]:
db("""
SELECT @most_films
""")

Executando query:
(42,)


In [44]:
ia.sender(answer='sql_ex06', task='views', question='ex06', answer_type='pyvar')

interactive(children=(Button(description='Enviar ex06', style=ButtonStyle()), Output()), _dom_classes=('widget…

## Operador `IN`

Suponha que desejamos listar todos os filmes dos 3 atores mais populares. Podemos começar listando os 3 atores mais populares:

In [45]:
db('''
SELECT 
    actor_id, first_name, last_name, COUNT(film_id) AS num_films
FROM
    actor
    INNER JOIN film_actor USING (actor_id)
GROUP BY 
    actor_id
ORDER BY 
    num_films DESC
LIMIT 3
''')

Executando query:
(107, 'GINA', 'DEGENERES', 42)
(102, 'WALTER', 'TORN', 41)
(198, 'MARY', 'KEITEL', 40)


Vamos criar uma tabela temporária para guardar a informação de `actor_id` desses atores:

In [46]:
db('DROP TABLE IF EXISTS temp_pop_actors')
db('''
CREATE TEMPORARY TABLE temp_pop_actors
    SELECT first_name, last_name, actor_id FROM
        actor
        INNER JOIN film_actor USING (actor_id)
    GROUP BY 
        actor_id
    ORDER BY 
        COUNT(film_id) DESC
    LIMIT 3
''')
db('''
SELECT * from temp_pop_actors
''')

Executando query:
Executando query:
Executando query:
('GINA', 'DEGENERES', 107)
('WALTER', 'TORN', 102)
('MARY', 'KEITEL', 198)


Por fim, vamos usar essa informação para listar os filmes dos atores populares:

In [47]:
db('''
SELECT DISTINCT
    title
FROM
    film
    INNER JOIN film_actor USING (film_id)
WHERE
    actor_id IN (SELECT actor_id FROM temp_pop_actors);
''')

Executando query:
('BED HIGHBALL',)
('CALENDAR GUNFIGHT',)
('CHAMBER ITALIAN',)
('CHAPLIN LICENSE',)
('CHARIOTS CONSPIRACY',)
('CLUELESS BUCKET',)
('COLDBLOODED DARLING',)
('CONEHEADS SMOOCHY',)
('DARKNESS WAR',)
('DEER VIRGINIAN',)
('DOGMA FAMILY',)
('ELEPHANT TROJAN',)
('EXCITEMENT EVE',)
('FRISCO FORREST',)
('GANDHI KWAI',)
('GOODFELLAS SALUTE',)
('GUNFIGHT MOON',)
('HALL CASSIDY',)
('HEARTBREAKERS BRIGHT',)
('HOOK CHARIOTS',)
('HYDE DOCTOR',)
('IMPACT ALADDIN',)
('INDIAN LOVE',)
('INTRIGUE WORST',)
('LICENSE WEEKEND',)
('LOUISIANA HARRY',)
('MAGNIFICENT CHITTY',)
('METAL ARMAGEDDON',)
('MIDNIGHT WESTWARD',)
('MOVIE SHAKESPEARE',)
('MUMMY CREATURES',)
('OPEN AFRICAN',)
('SEARCHERS WAIT',)
('SEVEN SWARM',)
('SIERRA DIVIDE',)
('SPIRITED CASUALTIES',)
('STORM HAPPINESS',)
('SUGAR WONKA',)
('TELEGRAPH VOYAGE',)
('TRAINSPOTTING STRANGERS',)
('WIFE TURN',)
('WINDOW SIDE',)
('AMELIE HELLFIGHTERS',)
('ARABIA DOGMA',)
('BANG KWAI',)
('CASABLANCA SUPER',)
('CASPER DRAGONFLY',)
('CROW GREASE',

Note o uso de *subqueries*!

Não se esqueça de limpar tudo no final!

In [48]:
db('DROP TABLE temp_pop_actors')

Executando query:


### Vamos praticar

Liste os atores (id, nome e sobrenome) que participaram dos 3 filmes mais rentáveis (aqueles que mais geraram receita para a locadora) ordenados pelo id do ator de modo crescente. Para isso, crie uma tabela temporária contendo o id do filme e a quantia total e use essa tabela para listar os atores.

In [49]:
db('DROP TABLE IF EXISTS temp_pop_films')
db('''
CREATE TEMPORARY TABLE temp_pop_films
    SELECT film_id, SUM(amount) AS total FROM
        film
        INNER JOIN inventory USING (film_id)
        INNER JOIN rental USING (inventory_id)
        INNER JOIN payment USING (rental_id)
    GROUP BY
        film_id
    ORDER BY
        total DESC
    LIMIT 3
''')
db('''
SELECT * from temp_pop_films
''')
db('''
SELECT DISTINCT
    actor_id, first_name, last_name
FROM
    actor
    INNER JOIN film_actor USING (actor_id)
WHERE
    film_id IN (SELECT film_id FROM temp_pop_films)
ORDER BY
    actor_id
''')

Executando query:
Executando query:
Executando query:
(879, Decimal('231.73'))
(973, Decimal('223.69'))
(1000, Decimal('214.69'))
Executando query:
(28, 'WOODY', 'HOFFMAN')
(47, 'JULIA', 'BARRYMORE')
(52, 'CARMEN', 'HUNT')
(107, 'GINA', 'DEGENERES')
(111, 'CAMERON', 'ZELLWEGER')
(138, 'LUCILLE', 'DEE')
(155, 'IAN', 'TANDY')
(157, 'GRETA', 'MALDEN')
(158, 'VIVIEN', 'BASINGER')
(166, 'NICK', 'DEGENERES')
(174, 'MICHAEL', 'BENING')
(178, 'LISA', 'MONROE')
(185, 'MICHAEL', 'BOLGER')
(200, 'THORA', 'TEMPLE')


In [50]:
sql_ex07 = ['''
CREATE TEMPORARY TABLE temp_pop_films
    SELECT film_id, SUM(amount) AS total FROM
        film
        INNER JOIN inventory USING (film_id)
        INNER JOIN rental USING (inventory_id)
        INNER JOIN payment USING (rental_id)
    GROUP BY
        film_id
    ORDER BY
        total DESC
    LIMIT 3
''',
'''
SELECT DISTINCT
    actor_id, first_name, last_name
FROM
    actor
    INNER JOIN film_actor USING (actor_id)
WHERE
    film_id IN (SELECT film_id FROM temp_pop_films)
ORDER BY
    actor_id
''']

ia.sender(answer='sql_ex07', task='views', question='ex07', answer_type='pyvar')

interactive(children=(Button(description='Enviar ex07', style=ButtonStyle()), Output()), _dom_classes=('widget…

## Subqueries

Os tópicos discutidos acima poderiam ser resolvidos, em grande parte, com subqueries. As subqueries são queries `SELECT` criadas dentro de outras queries. 

Poderíamos ter usado subqueries nos mesmos lugares onde usamos tabelas temporárias, nos tópicos acima. Quando a subquery pode ser transformada em uma tabela temporária independente, separada da query exterior, dizemos que a subquery é **não-correlacionada** com a query exterior.

Usar subqueries não-correlacionadas é um tópico controverso: podemos sempre usar uma tabela temporária ou, às vezes, pensar em um `JOIN` simples. Aliás, muitas vezes o otimizador de queries do banco de dados transformará a subquery em `JOIN`, se isso for vantajoso em termos de desempenho.

Uma subquery que depende da query externa (e portanto não pode ser separada em uma tabela temporária independente) é chamada de **subquery correlacionada**. Nestes casos podemos ter que executar a subquery para cada linha da query exterior! 

### Vamos praticar

Vamos refazer a atividade dos atores dos filmes mais rentáveis, usando subqueries. 

Temos um problema: o MySQL não suporta ``LIMIT`` em subqueries com o operador ``IN``. Vamos investigar isso mais de perto. 

Em primeiro lugar faça uma tradução direta da implementação da atividade anterior trocando tabela temporária por subquery.

In [51]:
try:
    db('''
    SELECT DISTINCT
        actor_id, first_name, last_name
    FROM
        actor
        INNER JOIN film_actor USING (actor_id)
    WHERE
        film_id IN (SELECT film_id FROM (
            SELECT film_id, SUM(amount) AS total FROM
                film
                INNER JOIN inventory USING (film_id)
                INNER JOIN rental USING (inventory_id)
                INNER JOIN payment USING (rental_id)
            GROUP BY
                film_id
            ORDER BY
                total DESC
            LIMIT 3
        ) AS temp_pop_films)
    ORDER BY
        actor_id
    ''')
except mysql.connector.ProgrammingError as e:
    print(f'ProgrammingError: {e}')

Executando query:
(28, 'WOODY', 'HOFFMAN')
(47, 'JULIA', 'BARRYMORE')
(52, 'CARMEN', 'HUNT')
(107, 'GINA', 'DEGENERES')
(111, 'CAMERON', 'ZELLWEGER')
(138, 'LUCILLE', 'DEE')
(155, 'IAN', 'TANDY')
(157, 'GRETA', 'MALDEN')
(158, 'VIVIEN', 'BASINGER')
(166, 'NICK', 'DEGENERES')
(174, 'MICHAEL', 'BENING')
(178, 'LISA', 'MONROE')
(185, 'MICHAEL', 'BOLGER')
(200, 'THORA', 'TEMPLE')


Ok, apareceu o problema. Mas considere que o problema original não precisava de IN desde o começo! Construa essa solução.

In [52]:
sql_ex08 = '''
SELECT DISTINCT
    actor_id, first_name, last_name
FROM
    actor
    INNER JOIN film_actor USING (actor_id)
    INNER JOIN (
        SELECT film_id, SUM(amount) AS total FROM
            film
            INNER JOIN inventory USING (film_id)
            INNER JOIN rental USING (inventory_id)
            INNER JOIN payment USING (rental_id)
        GROUP BY
            film_id
        ORDER BY
            total DESC
        LIMIT 3
    ) AS temp_pop_films USING (film_id)
ORDER BY
    actor_id
'''

db(sql_ex08)

Executando query:
(28, 'WOODY', 'HOFFMAN')
(47, 'JULIA', 'BARRYMORE')
(52, 'CARMEN', 'HUNT')
(107, 'GINA', 'DEGENERES')
(111, 'CAMERON', 'ZELLWEGER')
(138, 'LUCILLE', 'DEE')
(155, 'IAN', 'TANDY')
(157, 'GRETA', 'MALDEN')
(158, 'VIVIEN', 'BASINGER')
(166, 'NICK', 'DEGENERES')
(174, 'MICHAEL', 'BENING')
(178, 'LISA', 'MONROE')
(185, 'MICHAEL', 'BOLGER')
(200, 'THORA', 'TEMPLE')


In [53]:
ia.sender(answer='sql_ex08', task='views', question='ex08', answer_type='pyvar')

interactive(children=(Button(description='Enviar ex08', style=ButtonStyle()), Output()), _dom_classes=('widget…

# `UNION`

Quando duas tabelas tem **EXATAMENTE** as mesmas colunas, podemos concatená-las e formar uma grande tabela unificada usando o operador `UNION`. Por exemplo: suponha que desejamos montar uma lista dos nomes e sobrenomes de todos os clientes E de todos os funcionários. Eis uma solução possível:

In [54]:
db('DROP TABLE IF EXISTS nomes_clientes')
db('''
CREATE TEMPORARY TABLE nomes_clientes 
    SELECT first_name, last_name FROM customer
''')

Executando query:
Executando query:


In [55]:
db('DESCRIBE nomes_clientes')
db('SELECT * FROM nomes_clientes LIMIT 5')

Executando query:
('first_name', 'varchar(45)', 'NO', '', None, 'NULL')
('last_name', 'varchar(45)', 'NO', '', None, 'NULL')
Executando query:
('MARY', 'SMITH')
('PATRICIA', 'JOHNSON')
('LINDA', 'WILLIAMS')
('BARBARA', 'JONES')
('ELIZABETH', 'BROWN')


In [56]:
db('DROP TABLE IF EXISTS nomes_staff')
db('''
CREATE TEMPORARY TABLE nomes_staff 
    SELECT first_name, last_name FROM staff
''')

Executando query:
Executando query:


In [57]:
db('DESCRIBE nomes_staff')
db('SELECT * FROM nomes_staff LIMIT 5')

Executando query:
('first_name', 'varchar(45)', 'NO', '', None, 'NULL')
('last_name', 'varchar(45)', 'NO', '', None, 'NULL')
Executando query:
('Mike', 'Hillyer')
('Jon', 'Stephens')


In [58]:
db('DROP TABLE IF EXISTS nomes_all')
db('''
CREATE TEMPORARY TABLE nomes_all 
    (SELECT * FROM nomes_staff)
    UNION 
    (SELECT * FROM nomes_clientes)
''')

Executando query:
Executando query:


In [59]:
db('DESCRIBE nomes_all')
db('SELECT * FROM nomes_all LIMIT 5')

Executando query:
('first_name', 'varchar(45)', 'NO', '', '', 'NULL')
('last_name', 'varchar(45)', 'NO', '', '', 'NULL')
Executando query:
('Mike', 'Hillyer')
('Jon', 'Stephens')
('MARY', 'SMITH')
('PATRICIA', 'JOHNSON')
('LINDA', 'WILLIAMS')


In [60]:
db('DROP TABLE IF EXISTS nomes_clientes')
db('DROP TABLE IF EXISTS nomes_staff')
db('DROP TABLE IF EXISTS nomes_all')

Executando query:
Executando query:
Executando query:


**Vamos praticar:** refaça o exemplo acima mas use *subqueries* ao invés de *temp tables*. Ordene de forma ascendente por first_name e last_name.

In [61]:
sql_ex09 = '''
SELECT
    first_name, last_name
FROM (
    SELECT first_name, last_name FROM staff
    UNION
    SELECT first_name, last_name FROM customer
) AS nomes
ORDER BY
    first_name, last_name
'''

# db(sql_ex09)

In [62]:
ia.sender(answer='sql_ex09', task='views', question='ex09', answer_type='pyvar')

interactive(children=(Button(description='Enviar ex09', style=ButtonStyle()), Output()), _dom_classes=('widget…

## Desafios!

Faça uma lista de filmes que tenham mais de dois atores cujo nome inicia com a mesma letra do título do filme! Apresente o nome e a quantidade de atores. Ordene pelo id do filme. Dica: Pesquie a função `LEFT`

In [63]:
sql_ex10 = '''
SELECT
    title, COUNT(actor_id) AS num_actors
FROM
    film
    INNER JOIN film_actor USING (film_id)
    INNER JOIN actor USING (actor_id)
WHERE
    LEFT(title, 1) = LEFT(first_name, 1)
GROUP BY
    film_id
HAVING
    num_actors > 2
ORDER BY
    film_id
'''

db(sql_ex10)

Executando query:
('CROW GREASE', 3)
('JEDI BENEATH', 3)
('SUBMARINE BED', 3)


In [64]:
ia.sender(answer='sql_ex10', task='views', question='ex10', answer_type='pyvar')

interactive(children=(Button(description='Enviar ex10', style=ButtonStyle()), Output()), _dom_classes=('widget…

Semana do "DAN HARRIS": liste os clientes (nome e sobrenome) que nunca assistiram um filme do ator "DAN HARRIS" ou que já assistiram mas a ultima vez em que assistiram um filme dele foi antes de '2005-06-01'. Ordene pelo nome e sobrenome do cliente.

In [94]:
# returned in 2005-06-01 or later
first_use_case = '''
SELECT
    customer.customer_id
FROM
    customer
    INNER JOIN rental USING (customer_id)
    INNER JOIN inventory USING (inventory_id)
    INNER JOIN film USING (film_id)
    INNER JOIN film_actor USING (film_id)
    INNER JOIN actor USING (actor_id)
WHERE
    actor.first_name = 'DAN'
    AND actor.last_name = 'HARRIS'
    AND rental.return_date >= '2005-06-01'
'''

# rented in 2005-06-01 or later
second_use_case = '''
SELECT
    customer.customer_id
FROM
    customer
    INNER JOIN rental USING (customer_id)
    INNER JOIN inventory USING (inventory_id)
    INNER JOIN film USING (film_id)
    INNER JOIN film_actor USING (film_id)
    INNER JOIN actor USING (actor_id)
WHERE
    actor.first_name = 'DAN'
    AND actor.last_name = 'HARRIS'
    AND rental.rental_date >= '2005-06-01'
'''

# rented before 2005-06-01 and have not returned yet
third_use_case = '''
SELECT
    customer.customer_id
FROM
    customer
    INNER JOIN rental USING (customer_id)
    INNER JOIN inventory USING (inventory_id)
    INNER JOIN film USING (film_id)
    INNER JOIN film_actor USING (film_id)
    INNER JOIN actor USING (actor_id)
WHERE
    actor.first_name = 'DAN'
    AND actor.last_name = 'HARRIS'
    AND rental.rental_date < '2005-06-01'
    AND rental.return_date IS NULL
'''

all_use_cases = f'''
{first_use_case}
UNION
{second_use_case}
UNION
{third_use_case}
'''

sql_ex11 = f'''
SELECT
    first_name, last_name
FROM
    customer
WHERE
    customer_id NOT IN ({all_use_cases})
ORDER BY
    first_name, last_name
'''

db(sql_ex11)

Executando query:
('AARON', 'SELBY')
('AGNES', 'BISHOP')
('ALAN', 'KAHN')
('ALBERT', 'CROUSE')
('ALICIA', 'MILLS')
('ALLAN', 'CORNISH')
('ALLEN', 'BUTTERFIELD')
('ALLISON', 'STANLEY')
('AMANDA', 'CARTER')
('AMY', 'LOPEZ')
('ANA', 'BRADLEY')
('ANDREA', 'HENDERSON')
('ANDREW', 'PURDY')
('ANGEL', 'BARCLAY')
('ANITA', 'MORALES')
('ANN', 'EVANS')
('ANNA', 'HILL')
('ANNE', 'POWELL')
('ANNETTE', 'OLSON')
('ANNIE', 'RUSSELL')
('ANTHONY', 'SCHWAB')
('ANTONIO', 'MEEK')
('ARMANDO', 'GRUBER')
('AUDREY', 'RAY')
('AUSTIN', 'CINTRON')
('BEATRICE', 'ARNOLD')
('BECKY', 'MILES')
('BEN', 'EASTER')
('BERNARD', 'COLBY')
('BESSIE', 'MORRISON')
('BETH', 'FRANKLIN')
('BILLIE', 'HORTON')
('BILLY', 'POULIN')
('BOB', 'PFEIFFER')
('BRAD', 'MCCURDY')
('BRANDY', 'GRAVES')
('BRENT', 'HARKINS')
('BRETT', 'CORNWELL')
('BRIAN', 'WYMAN')
('BRITTANY', 'RILEY')
('BYRON', 'BOX')
('CALVIN', 'MARTEL')
('CARLA', 'GUTIERREZ')
('CAROLE', 'BARNETT')
('CARRIE', 'PORTER')
('CASEY', 'MENA')
('CATHY', 'SPENCER')
('CECIL', 'VINES')
(

In [95]:
ia.sender(answer='sql_ex11', task='views', question='ex11', answer_type='pyvar')

interactive(children=(Button(description='Enviar ex11', style=ButtonStyle()), Output()), _dom_classes=('widget…

- Faça uma consulta que retorne, para cada ator, a seguinte informação:

| first_name | last_name | filmes por categoria |
|--|--|--|
| PENELOPE | GUINESS | Animation: ANACONDA CONFESSIONS; Children: LANGUAGE COWBOY; Classics: COLOR PHILADELPHIA, WESTWARD SEABISCUIT; Comedy: VERTIGO NORTHWEST; Documentary: ACADEMY DINOSAUR; Family: KING EVOLUTION, SPLASH GUMP; Foreign: MULHOLLAND BEAST; Games: BULWORTH COMMANDMENTS, HUMAN GRAFFITI; Horror: ELEPHANT TROJAN, LADY STAGE, RULES HUMAN; Music: WIZARD COLDBLOODED; New: ANGELS LIFE, OKLAHOMA JUMANJI; Sci-Fi: CHEAPER CLYDE; Sports: GLEAMING JAWBREAKER |
| NICK | WAHLBERG | Action: BULL SHAWSHANK; Animation: FIGHT JAWBREAKER; Children: JERSEY SASSY; Classics: DRACULA CRYSTAL, GILBERT PELICAN; Comedy: MALLRATS UNITED, RUSHMORE MERMAID; Documentary: ADAPTATION HOLES; Drama: WARDROBE PHANTOM; Family: APACHE DIVINE, CHISUM BEHAVIOR, INDIAN LOVE, MAGUIRE APACHE; Foreign: BABY HALL, HAPPINESS UNITED; Games: ROOF CHAMPION; Music: LUCKY FLYING; New: DESTINY SATURDAY, FLASH WARS, JEKYLL FROGMEN, MASK PEACH; Sci-Fi: CHAINSAW UPTOWN, GOODFELLAS SALUTE; Travel: LIAISONS SWEET, SMILE EARRING |
| etc | etc | etc |

Ordene pelo nome e sobrenome do ator.

Dica: use `GROUP_CONCAT` para agrupar todas as strings de uma coluna em uma string só, e `CONCAT` para unir strings particulares.

In [68]:
sql_ex12 = '''
-- SUA QUERY AQUI!
'''

db(sql_ex12)

Executando query:


In [69]:
ia.sender(answer='sql_ex12', task='views', question='ex12', answer_type='pyvar')

interactive(children=(Button(description='Enviar ex12', style=ButtonStyle()), Output()), _dom_classes=('widget…

## Conclusão

Façamos uma pausa para apreciar quão longe estamos: já conseguimos criar nossas tabelas, inserir informação, removê-la, atualizá-la, e consultar nossa base de maneiras bem sofisticadas! Vimos desde `SELECT` simples até buscas mais complexas envolvendo várias etapas de processamento para obter o dado desejado.

Por hoje é só, feche sua conexão e bom descanso!

In [70]:
# connection.close()

## Conferir Notas

Confira se as notas na atividade são as esperadas!

Primeiro na atividade atual!

In [71]:
ia.grades(by="task", task="views")

|    | Tarefa   |    Nota |
|---:|:---------|--------:|
|  0 | views    | 8.33333 |

In [72]:
ia.grades(task="views")

|    | Atividade   | Exercício   |   Peso |   Nota |
|---:|:------------|:------------|-------:|-------:|
|  0 | views       | ex01        |      1 |     10 |
|  1 | views       | ex02        |      1 |     10 |
|  2 | views       | ex03        |      1 |     10 |
|  3 | views       | ex04        |      1 |     10 |
|  4 | views       | ex05        |      1 |     10 |
|  5 | views       | ex06        |      1 |     10 |
|  6 | views       | ex07        |      1 |     10 |
|  7 | views       | ex08        |      1 |     10 |
|  8 | views       | ex09        |      1 |     10 |
|  9 | views       | ex10        |      1 |     10 |
| 10 | views       | ex11        |      1 |      0 |
| 11 | views       | ex12        |      1 |      0 |

In [73]:
ia.grades(by="task")

|    | Tarefa       |     Nota |
|---:|:-------------|---------:|
|  0 | agg_join     | 10       |
|  1 | ddl          | 10       |
|  2 | dml          | 10       |
|  3 | group_having | 10       |
|  4 | newborn      | 10       |
|  5 | select01     | 10       |
|  6 | sql_review1  |  0       |
|  7 | views        |  8.33333 |